### Mutations associated with resistance to Paxlovid treatment

Here we investigate the global trends in mutations in the SARS-CoV-2 genome associated with resistance to Paxlovid treatment. We used [Iketani *et al.*](https://www.nature.com/articles/s41586-022-05514-2) as reference for the mutations. Global SARS-CoV-2 genome sequencing data was downloaded from [GISAID](https://gisaid.org/), and the processing carried out in Python. The code is given below

### Import libraries, version check

In [3]:
# code written by Jonathan D. Ip on 30-Dec-2022 on a Linux based system
# verified on a Windows 10 based system on 01-Jan-2023 and a
# Ubuntu 22.04.1 LTS system on 03-Jan-2023 by S. M. Umer Abdullah 
# For queries please contact jdip1007@connect.hku.hk

print('This code has been written and tested using the following libraries:')
print('Python v3.7.5 (default, Dec  9 2021, 17:04:37)')
print('[GCC 7.5.0]')
print('pandas v1.3.5')
print('numpy v1.21.6')
print('json v2.0.9')
print('openpyxl v3.0.10')
print('\n')
print('Libraries installed on this system are:')
import os
import sys
print('Python v%s' % sys.version)
import pandas as pd
print('pandas v%s' % pd.__version__)
#from tqdm.auto import tqdm
#import tqdm as tq
#print('tqdm v%s' % tq.__version__)
import numpy as np
print('numpy v%s' % np.__version__)
import json
print('json v%s' % json.__version__)
import openpyxl
print('openpyxl v%s' % openpyxl.__version__)
import glob

This code has been written and tested using the following libraries:
Python v3.7.5 (default, Dec  9 2021, 17:04:37)
[GCC 7.5.0]
pandas v1.3.5
numpy v1.21.6
json v2.0.9
openpyxl v3.0.10


Libraries installed on this system are:
Python v3.7.5 (default, Dec  9 2021, 17:04:37) 
[GCC 8.4.0]
pandas v1.3.5
numpy v1.21.6
json v2.0.9
openpyxl v3.0.10


### Set input and output directories

In [2]:
input_folder  = "../data/" ## input folder
output_folder  = "../output/" ## output folder
if(not os.path.isdir(output_folder)): ## check if output folder exists
    os.mkdir(output_folder) ## create output folder if it does not exist

### Initialize variables

In [4]:
columns = ["Collection date", "Type", "Location", "Pango lineage", "AA Substitutions", "Is complete?", "Host",\
           "Passage details/history", "Accession ID"]

ANY = "__ANY__" ## If the mutation list contains ANY as its first element, the strain containing any one of the included
# mutations is considered

## Mutations listed in Iketani et al.
target_mut = [
    
    ["NSP5_T21I"], 
    ["NSP5_P252L"], 
    ["NSP5_T304I"],
    ["NSP5_E166V"], 
    ["NSP5_L50F"], 
    ["NSP5_S144A"],
    ["NSP5_A173V"],
    ["NSP5_T21I", "NSP5_E166V"], 
    ["NSP5_T21I", "NSP5_S144A"], 
    ["NSP5_L50F", "NSP5_E166V"], 
    ["NSP5_T21I", "NSP5_A173V", "NSP5_T304I"],
    [ANY, "NSP5_T21I","NSP5_P252L","NSP5_T304I","NSP5_E166V","NSP5_S144A"], ### Any of the mutations starting from the second element is considered
    [ANY, "NSP5_T21I","NSP5_P252L","NSP5_T304I","NSP5_E166V","NSP5_L50F","NSP5_S144A","NSP5_A173V"],
    
## Mutations listed in other publications
    ["NSP5_G15S"],
    ["NSP5_T45I"],
    ["NSP5_D48Y"],
    ["NSP5_M49T"],
    ["NSP5_M49L"],
    ["NSP5_M49I"],
    ["NSP5_M49V"],
    ["NSP5_Y54A"],
    ["NSP5_Y54C"],
    ["NSP5_T135I"], 
    ["NSP5_F140A"],
    ["NSP5_F140L"],
    ["NSP5_S144E"],
    ["NSP5_S144T"],
    ["NSP5_S144M"],
    ["NSP5_S144F"],
    ["NSP5_S144G"],
    ["NSP5_S144Y"],
    ["NSP5_H164N"],
    ["NSP5_M165T"],
    ["NSP5_E166A"],
    ["NSP5_E166G"],
    ["NSP5_E166V"],
    ["NSP5_L167F"],
    ["NSP5_P168del"],
    ["NSP5_H172Y"],
    ["NSP5_H172Q"], 
    ["NSP5_H172F"],
    ["NSP5_A173T"],
    ["NSP5_A173V"],
    ["NSP5_V186G"],
    ["NSP5_Q189K"],
    ["NSP5_Q192L"],
    ["NSP5_Q192P"],
    ["NSP5_Q192T"],
    ["NSP5_Q192S"],
    ["NSP5_Q192A"],
    ["NSP5_Q192I"],
    ["NSP5_Q192H"],
    ["NSP5_Q192V"],
    ["NSP5_Q192W"],
    ["NSP5_Q192C"],
    ["NSP5_Q192F"],
    ["NSP5_D248E"],

]

any_of_all_mutations = [ANY,] + [ m[0] for m in target_mut if len(m) == 1 ]

any_of_Affect_3CLpro_activity_affecting_mutations = [
    ANY,
    "NSP5_G15S",
    "NSP5_Y54A",
    "NSP5_T135I",
    "NSP5_F140A",
    "NSP5_F140L",
    "NSP5_S144E",
    "NSP5_S144G",
    "NSP5_S144M",
    "NSP5_S144Y",
    "NSP5_S144T",
    "NSP5_H164N",
    "NSP5_M165T",
    "NSP5_E166G",
    "NSP5_H172Q",
    "NSP5_H172F",
    "NSP5_V186G",
    "NSP5_Q189K",
    "NSP5_Q192T",
    "NSP5_Q192S",
    "NSP5_Q192A",
    "NSP5_Q192I",
    "NSP5_Q192H",
    "NSP5_Q192V",
    "NSP5_Q192W",
    "NSP5_Q192C",
    "NSP5_Q192F",
    "NSP5_D248E"
]

any_of_Reduction_of_SARS_CoV_2_replication_AND_mutations = [
    ANY,
    "NSP5_S144A",
    "NSP5_E166A",
    "NSP5_H172Y",
    "NSP5_A173V",
    "NSP5_Q192L",
    "NSP5_Q192P",
]

any_of_Reduction_of_SARS_CoV_2_replication = [
    ANY,
    "NSP5_T21I",
    "NSP5_T45I",
    "NSP5_D48Y",
    "NSP5_M49I",
    "NSP5_M49L",
    "NSP5_M49T",
    "NSP5_M49V",
    "NSP5_L50F",
    "NSP5_Y54C",
    "NSP5_S144A",
    "NSP5_E166V",
    "NSP5_L167F",
    "NSP5_P168del",
    "NSP5_A173T",
    "NSP5_A173V",
    "NSP5_P252L",
    "NSP5_T304I",
]

target_mut.append(any_of_all_mutations)
target_mut.append(any_of_Affect_3CLpro_activity_affecting_mutations)
target_mut.append(any_of_Reduction_of_SARS_CoV_2_replication_AND_mutations)

target_lineage = [
    "B.1.1.318", ## includes AZ
    "B.1.1.7", ## VOC Alpha
    "B.1.1.529", ## VOC Omicron
    "B.1.351", ## VOC Beta
    "B.1.1.28.1", ## VOC Gamma
    "B.1.617.2", ## VOC Delta
    "B.1.1.529.5.3.1.1.1.1.1", ## VOC Omicron sublineage BQ.1
    "XBB", ## VOC Omicron sublineage XBB
    "B.1.1.529.1", ## VOC Omicron sublineage BA.1
    "B.1.1.529.2", ## VOC Omicron sublineage BA.2
    "B.1.1.529.4", ## VOC Omicron sublineage BA.4
    "B.1.1.529.5", ## VOC Omicron sublineage BA.5
    "B.1.1.529.2.75", ## VOC Omicron sublineage BA.2.75
    "B.1.1.1", ## Has G15S as defining mutation
]

groupby_period = 'Collection Quarter' ## Group sequences by quarter of collection

### Read data and filter sequences based on completeness

In [5]:
total, complete = 0, 0 # keep count of total sequences, and sequences which fit our criterion of being complete
res_df = pd.DataFrame()
# parse over input file taking in 1e4 sequences in each pass
for chunk in tqdm(pd.read_csv(input_folder+"metadata.tsv", header=0, sep='\t', chunksize=10000)):
    chunk = chunk[columns]
    total += len(chunk)
    chunk = chunk[chunk["Is complete?"] == True]
    chunk = chunk[chunk["Host"] == "Human"]
    chunk = chunk[chunk["Type"] == "betacoronavirus"]
    chunk = chunk[chunk["Passage details/history"] == "Original"]
    chunk = chunk[["Collection date", "Location", "Pango lineage", "AA Substitutions", "Accession ID"]]
    complete += len(chunk)
    #print(total, complete)
    res_df = pd.concat([res_df, chunk])

### Print total number of sequences, and number of complete sequences

In [6]:
print('Total number of sequences in the GISAID metadata file: '+str(total))
print('Number of complete sequences in the GISAID metadata file: '+str(complete))

Total number of sequences in the GISAID metadata file: 14157266
Number of complete sequences in the GISAID metadata file: 13770074


### Filter sequences with complete collection date

In [9]:
# only keep complete collection dates with year, month, and day
def reformat_date(date_strings):
    if date_strings.count('-') == 2: # date value with 2 hyphens means complete day, month, and year
        return date_strings
    else:
        return np.nan
    
res_df["Collection date"] = res_df["Collection date"].apply(reformat_date)
res_df.dropna(subset=["Collection date",], inplace=True)

### Print final number of sequences used for subsequent processing

In [10]:
print('Number of sequences with complete date of collection: '+str(len(res_df)))

Number of sequences with complete date of collection: 13446588


### Group dates by week, quarter, and month

In [11]:
res_df["Collection date"] = pd.to_datetime(res_df["Collection date"], format='%Y/%m/%d')
res_df["Collection Week"] = res_df["Collection date"].dt.to_period(freq = 'W')  
res_df["Collection Quarter"] = res_df["Collection date"].dt.to_period(freq = 'Q')  
res_df["Collection Month"] = res_df["Collection date"].dt.to_period(freq = 'M')  

res_df.sort_values(by=[groupby_period], inplace=True)
res_df["Accession ID"].to_csv(output_folder+"accession_id.txt", index=False)

D:\Installed_1\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  import sys


### Output the counts of mutations associated with resistance to paxlovid

In [ ]:
target_mut_freq = {}
Dates = []
# parse over all mutations of interest
for tmuts in tqdm(target_mut):
    if tmuts[0] == ANY:
        tmut_name = "Any of "+",".join(tmuts[1:])
    else:
        tmut_name = ",".join(tmuts)
        
    target_mut_freq[tmut_name] = []
    for name, group_df in res_df.groupby([groupby_period]):
        target_mut_freq[tmut_name].append(0)
        for muts in group_df["AA Substitutions"]:
            if tmuts[0] != ANY:
                if all([tmut in muts for tmut in tmuts]):
                    target_mut_freq[tmut_name][-1] += 1
            else:
                if any([tmut in muts for tmut in tmuts[1:]]):
                    target_mut_freq[tmut_name][-1] += 1
        
seq_count = []
for name, group_df in tqdm(res_df.groupby([groupby_period])):
    Dates.append(name.to_timestamp().date())
    seq_count.append(len(group_df))
    
df = pd.DataFrame(target_mut_freq)
df.index = Dates
df["Sequence count"] = seq_count
df.loc['Total']= df.sum()
df.to_csv(output_folder+"Paxlovid_mutation.csv", sep=',', index=True)

### Group mutations appearing in individual continents and countries

In [ ]:
def get_Continent(location):
    return location.split('/')[0].strip()

def get_Country(location):
    return location.split('/')[1].strip()

res_df["Continent"] = res_df["Location"].apply(get_Continent)
res_df["Country"] = res_df["Location"].apply(get_Country)

### Output the counts of mutations appearing in individual continents

In [ ]:
prevalence_columns = []
# parse over all mutations of interest
for tmuts in tqdm(target_mut):
    
    tmp_df = res_df.copy()
    
    if tmuts[0] == ANY:
        tmut_name = "Any of "+",".join(tmuts[1:])
        tmp_df = tmp_df[tmp_df["AA Substitutions"].str.contains('|'.join(tmuts[1:]))]
    else:
        tmut_name = ",".join(tmuts)
        for tmut in tmuts:
            tmp_df = tmp_df[tmp_df['AA Substitutions'].str.contains(tmut)]
        
    tmp_series = tmp_df["Continent"].value_counts()
    tmp_series = tmp_series.rename(tmut_name)
    prevalence_columns.append(tmp_series)
    
prevalence_continent_df = pd.concat(prevalence_columns,axis=1)
prevalence_continent_df["Sequence count"] = res_df["Continent"].value_counts()
prevalence_continent_df.to_csv(output_folder+"Continent_prevalence.csv", sep=',', index=True)

### Output the counts of mutations appearing in individual countries

In [ ]:
prevalence_columns = []
# parse over all mutations of interest
for tmuts in tqdm(target_mut):
    tmp_df = res_df.copy()
    
    if tmuts[0] == ANY:
        tmut_name = "Any of "+",".join(tmuts[1:])
        tmp_df = tmp_df[tmp_df["AA Substitutions"].str.contains('|'.join(tmuts[1:]))]
    else:
        tmut_name = ",".join(tmuts)
        for tmut in tmuts:
            tmp_df = tmp_df[tmp_df['AA Substitutions'].str.contains(tmut)]
            
    tmp_series = tmp_df["Country"].value_counts()
    tmp_series = tmp_series.rename(tmut_name)
    prevalence_columns.append(tmp_series)
    
prevalence_country_df = pd.concat(prevalence_columns,axis=1)
prevalence_country_df["Sequence count"] = res_df["Country"].value_counts()
tmp_df = None
prevalence_country_df.to_csv(output_folder+"Country_prevalence.csv", sep=',', index=True)

### Output the counts of mutations appearing in specific lineages

In [ ]:
prevalence_columns = []
# parse over all mutations of interest
for tmuts in tqdm(target_mut):
    tmp_df = res_df.copy()

    if tmuts[0] == ANY:
        tmut_name = "Any of "+",".join(tmuts[1:])
        tmp_df = tmp_df[tmp_df["AA Substitutions"].str.contains('|'.join(tmuts[1:]))]
    else:
        tmut_name = ",".join(tmuts)
        for tmut in tmuts:
            tmp_df = tmp_df[tmp_df['AA Substitutions'].str.contains(tmut)]
    
    tmp_series = tmp_df["Pango lineage"].value_counts()
    tmp_series = tmp_series.rename(tmut_name)
    prevalence_columns.append(tmp_series)
    
prevalence_lineage_df = pd.concat(prevalence_columns,axis=1)
prevalence_lineage_df["Sequence count"] = res_df["Pango lineage"].value_counts()
prevalence_lineage_df.to_csv(output_folder+f"Lineage_prevalence.csv", sep=',', index=True)

### Convert lineages into the original notation to easily incorporate sublineages

In [ ]:
alias = {}
 
# Opening JSON file
with open(input_folder+'alias_key.json') as json_file: ## https://github.com/cov-lineages/pango-designation/blob/master/pango_designation/alias_key.json
    alias = json.load(json_file)
    
def convert_lineages(lineage):
    if lineage.split('.')[0] in alias:
        return lineage.replace(lineage.split('.')[0], alias[lineage.split('.')[0]])
    return lineage

### Break down lineages into the most basic form for easy selection
res_df["Detailed lineage"] = res_df["Pango lineage"].apply(convert_lineages)

### Extract the accession numbers of the 13 strains with NSP5_E166V

In [ ]:
for accid, lineage, mut in zip(res_df["Accession ID"], res_df["Detailed lineage"], res_df['AA Substitutions']):
    if "NSP5_E166V" in mut:
        print(accid, lineage)

### Extract the accession numbers strains with both E166V and L50F

In [ ]:
for accid, lineage, mut in tqdm(zip(res_df["Accession ID"], res_df["Detailed lineage"], res_df['AA Substitutions'])):
    if "NSP5_E166V" in mut and "NSP5_L50F" in mut:
        print(accid, lineage)

### Combine all the output CSVs into one xlsx file.

In [ ]:
#import glob 
#!pip install xlsxwriter
#!pip install openpyxl
#import openpyxl, os

if os.path.exists(output_folder+'database.xlsx'):
    os.remove(output_folder+'database.xlsx')

openpyxl.Workbook().save(output_folder+'database.xlsx')
writer = pd.ExcelWriter(output_folder+'database.xlsx', engine='xlsxwriter')

f = output_folder+"Lineage_trend.csv"
df = pd.read_csv(f)
df.to_excel(writer, index=False, sheet_name=f"Lineage trend (raw)")

f = output_folder+"Lineage_prevalence.csv"
df = pd.read_csv(f)
df.to_excel(writer, index=False, sheet_name=f"Lineage prev. (raw)")

f = output_folder+"Paxlovid_mutation.csv"
df = pd.read_csv(f)
df.to_excel(writer, index=False, sheet_name=f"Paxlovid mutation (raw)")

f = output_folder+"Continent_prevalence.csv"
df = pd.read_csv(f)
df.to_excel(writer, index=False, sheet_name=f"Continent prev. (raw)")

f = output_folder+"Country_prevalence.csv"
df = pd.read_csv(f)
df.to_excel(writer, index=False, sheet_name=f"Country prev. (raw)")

writer.save()

### Count number of strains containing the 3CLpro inhibitor affecting mutations

In [ ]:
Total_number_of_sequences_with_mutations = 0 
with open(output_folder+'accession_number_with_mutations.txt', 'w') as fw:
    for accid, muts in tqdm(zip(res_df["Accession ID"], res_df['AA Substitutions'])):
        if any([tmut in muts for tmut in any_of_all_mutations[1:]]):
            fw.write(accid+'\n')
            Total_number_of_sequences_with_mutations += 1
print(f"Total_number_of_sequences_with_mutations:: {Total_number_of_sequences_with_mutations}")

### Export a table containing information of the strains carrying the 3CLpro inhibitor affecting mutations

In [ ]:
accession_id = []
location     = []
collection_d = []

for accid, muts, cdate, location in tqdm(zip(res_df["Accession ID"], res_df['AA Substitutions'], res_df['Collection date'], res_df['Location'])):
    if any([tmut in muts for tmut in any_of_all_mutations[1:]]):
        accession_id.append()
        location.append()
        collection_d.append()

pd.DataFrame({
    'Accession ID': accession_id,
    'Location': location,
    'Collection Date': collection_d,
}).to_csv(output_folder+"strain_info_table.csv", sep=',', index=False)